In [298]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, 7, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)

print(latest_file)

graph_list/Town05\5_graph_v3.adjlist


In [299]:
################# Change start, end point #################

reverse = input("Do reverse search or not? (y for reverse search, other for original search)\n")

start, end = (2639, 572), (3242, 527)

if reverse == "y":
    start, end = end, start


In [300]:
if start == (0,0) or end == (0,0):
    print("Didn't change to your point!!!!!!!!")

all_whites_pos = iter_all_white_points(waypoint_map)

new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(255, 0, 0), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(0, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "", False)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    if reverse == "y":
        fp = final_path[0][::-1]
        fp = [fp]
        add_path(G, fp, new_file_name)
    else:
        add_path(G, final_path, new_file_name)
    
    print(new_file_name, "Store!")
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", rdp_img)

Start point: (3242, 527)
End point: (2639, 572)
Finding Path...
Get!!!
found!!!
[(3242, 527), (3238, 527), (3234, 527), (3230, 527), (3226, 526), (3222, 526), (3218, 526), (3214, 526), (3210, 526), (3206, 526), (3202, 526), (3198, 526), (3194, 526), (3190, 526), (3186, 526), (3182, 526), (3178, 526), (3174, 526), (3170, 526), (3166, 526), (3162, 526), (3158, 526), (3154, 526), (3150, 526), (3146, 526), (3142, 526), (3138, 526), (3134, 526), (3130, 526), (3126, 525), (3122, 525), (3118, 525), (3114, 525), (3110, 525), (3106, 525), (3102, 525), (3098, 525), (3094, 525), (3090, 525), (3086, 525), (3082, 525), (3078, 525), (3074, 525), (3070, 525), (3066, 525), (3062, 525), (3058, 525), (3054, 525), (3050, 525), (3046, 525), (3042, 525), (3038, 525), (3034, 525), (3030, 524), (3026, 524), (3022, 524), (3018, 524), (3014, 524), (3010, 524), (3006, 524), (3002, 524), (2998, 524), (2994, 524), (2990, 524), (2986, 524), (2982, 524), (2978, 524), (2974, 524), (2970, 524), (2966, 524), (2962, 52

In [302]:
############### Quick test for other start point with the same goal point
start, end =  (2622, 853), (3242, 527)


fp = nx.shortest_path(G,  start,  end)
draw_img = draw_waypoint(waypoint_map, start, end, fp)
rdp_img, _ = rdp_algorithm(draw_img, fp)
cv2.imwrite(f"test/check__{start}_{end}.png", rdp_img)
print(fp)

[(2622, 853), (2626, 853), (2627, 847), (2627, 835), (2627, 831), (2627, 819), (2623, 818), (2627, 807), (2627, 803), (2628, 791), (2628, 787), (2628, 775), (2628, 771), (2628, 759), (2628, 755), (2628, 743), (2628, 739), (2628, 727), (2628, 723), (2628, 711), (2628, 707), (2628, 695), (2628, 691), (2628, 679), (2628, 675), (2628, 663), (2628, 659), (2628, 647), (2628, 643), (2628, 630), (2629, 625), (2630, 620), (2631, 615), (2634, 605), (2636, 600), (2643, 593), (2641, 591), (2637, 585), (2639, 581), (2639, 572), (2645, 570), (2647, 567), (2655, 556), (2658, 553), (2667, 545), (2672, 542), (2688, 534), (2694, 531), (2709, 526), (2714, 524), (2727, 522), (2731, 522), (2743, 522), (2747, 522), (2759, 522), (2763, 522), (2775, 522), (2779, 522), (2791, 522), (2795, 522), (2807, 522), (2811, 522), (2823, 522), (2827, 522), (2838, 523), (2842, 523), (2854, 523), (2858, 523), (2870, 523), (2874, 523), (2886, 523), (2890, 523), (2902, 523), (2906, 523), (2918, 523), (2922, 523), (2934, 524)